# Model registry: MLFlow

Example on how to use the MLFlow model during the training of a model

## 1. Install MLFlow

In [1]:
!pip install -r requirements.txt
!pip freeze | grep mlflow

"grep" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.



## 2. Start the MLFlow server.

To do this, run the following command in a terminal:

mlflow ui --host 0.0.0.0 --port 5001

## 3. Create a new experiment

In [2]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5001")
mlflow.set_experiment("mlflow-model-training-iris")

2025/03/27 10:02:46 INFO mlflow.tracking.fluent: Experiment with name 'mlflow-model-training-iris' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/725460823759344054', creation_time=1743066166767, experiment_id='725460823759344054', last_update_time=1743066166767, lifecycle_stage='active', name='mlflow-model-training-iris', tags={}>

## 4. Log a model experiment result

In [3]:
# Lets first create a simple function to train a model
# The function will receive a set of hyperparameters and return the model and the accuracy
import datetime
from mlflow.models import infer_signature

def get_data():
    from sklearn import datasets

    # X, y = datasets.make_classification(n_samples=1000, n_features=4, n_informative=4, n_redundant=0, n_classes=3, n_clusters_per_class=1, class_sep=0.5, random_state=40)
    # use iris dataset
    X, y = datasets.load_iris(return_X_y=True)
    return X, y

def train_model(hyperparameters):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    import mlflow.sklearn
    X, y = get_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)
    model = RandomForestClassifier(**hyperparameters)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    signature = infer_signature(X, model.predict(X))
    accuracy = accuracy_score(y_pred, y_test)
    return model, signature, accuracy

# Now lets train a model with some hyperparameters
hyperparameters = {"n_estimators": 10, "max_depth": 5}
model, signature, accuracy = train_model(hyperparameters)

# Now lets log the model and the accuracy
date = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
with mlflow.start_run(run_name=f"mlflow-training-{date}"):
    
    # Hyperparameters are logged as parameters
    for hyperparameter, value in hyperparameters.items():
        mlflow.log_param(hyperparameter, value)
    
    # Accuracy is logged as a metric 
    mlflow.log_metric("accuracy", accuracy)
    
    # Model is logged as an artifact
    mlflow.sklearn.log_model(model, "model", signature=signature)

🏃 View run mlflow-training-2025-03-27-10-03-02 at: http://localhost:5001/#/experiments/725460823759344054/runs/fb9f264c3b75482fb70aec50801a3694
🧪 View experiment at: http://localhost:5001/#/experiments/725460823759344054


## 5. Run a hyperparameter search and log the results

In [4]:

import optuna
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score


X, y = get_data()

def optimize_rf(trial):
    
    with mlflow.start_run(run_name=f"optuna-hp-{trial.number}-{datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')}"):
        # Set the hyperparameter values that we want to optimize
        n_estimators = trial.suggest_int('n_estimators', 1, 100)
        max_depth = trial.suggest_int('max_depth', 2, 10)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
        max_features = trial.suggest_float('max_features', 0.1, 1.0)
        
        
        # Create a random forest classifier using the suggested hyperparameters
        rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                    min_samples_split=min_samples_split,
                                    min_samples_leaf=min_samples_leaf,
                                    max_features=max_features)
        
        # Use cross-validation to evaluate the performance of the classifier
        scores = cross_val_score(rf, X, y, cv=5)
        
        # Log the hyperparameters and cross-validation scores to MLflow
        mlflow.log_param('n_estimators', n_estimators)
        mlflow.log_param('max_depth', max_depth)
        mlflow.log_param('min_samples_split', min_samples_split)
        mlflow.log_param('min_samples_leaf', min_samples_leaf)
        mlflow.log_param('max_features', max_features)
        mlflow.log_metric('mean_cv_score', scores.mean())
    
    # Return the mean of the cross-validation scores as the objective value
    return scores.mean()

# Create an Optuna study
study = optuna.create_study()

# Run the optimization loop
study.optimize(optimize_rf, n_trials=50)

# Get the best hyperparameter values
best_params = study.best_params

c:\Users\Usuario\Documents\Master IA\MIA 24-25\EDEM_MDA2425\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-27 10:09:01,482] A new study created in memory with name: no-name-103ed5c2-c305-4686-a163-3dac94a5cd3e
[I 2025-03-27 10:09:03,781] Trial 0 finished with value: 0.9666666666666668 and parameters: {'n_estimators': 12, 'max_depth': 4, 'min_samples_split': 8, 'min_samples_leaf': 13, 'max_features': 0.5608613253286324}. Best is trial 0 with value: 0.9666666666666668.


🏃 View run optuna-hp-0-2025-03-27-10-09-01 at: http://localhost:5001/#/experiments/725460823759344054/runs/95a80ae4df2a4837b70e48e42bf2b5ee
🧪 View experiment at: http://localhost:5001/#/experiments/725460823759344054


[I 2025-03-27 10:09:04,240] Trial 1 finished with value: 0.9466666666666665 and parameters: {'n_estimators': 60, 'max_depth': 7, 'min_samples_split': 16, 'min_samples_leaf': 9, 'max_features': 0.6304701514111404}. Best is trial 1 with value: 0.9466666666666665.


🏃 View run optuna-hp-1-2025-03-27-10-09-03 at: http://localhost:5001/#/experiments/725460823759344054/runs/fb808840c609477a947cc35836b028f4
🧪 View experiment at: http://localhost:5001/#/experiments/725460823759344054


[I 2025-03-27 10:09:04,896] Trial 2 finished with value: 0.9466666666666665 and parameters: {'n_estimators': 98, 'max_depth': 8, 'min_samples_split': 12, 'min_samples_leaf': 19, 'max_features': 0.8380380964105396}. Best is trial 1 with value: 0.9466666666666665.


...

🏃 View run optuna-hp-47-2025-03-27-10-09-56 at: http://localhost:5001/#/experiments/725460823759344054/runs/5457f00df0ad421e8b78758c9f3eb33f
🧪 View experiment at: http://localhost:5001/#/experiments/725460823759344054


[I 2025-03-27 10:09:57,221] Trial 48 finished with value: 0.9533333333333334 and parameters: {'n_estimators': 6, 'max_depth': 9, 'min_samples_split': 20, 'min_samples_leaf': 8, 'max_features': 0.3538813433981539}. Best is trial 47 with value: 0.9066666666666666.


🏃 View run optuna-hp-48-2025-03-27-10-09-56 at: http://localhost:5001/#/experiments/725460823759344054/runs/1b50709264274af19589ecf072f91323
🧪 View experiment at: http://localhost:5001/#/experiments/725460823759344054


[I 2025-03-27 10:09:57,868] Trial 49 finished with value: 0.9466666666666665 and parameters: {'n_estimators': 14, 'max_depth': 7, 'min_samples_split': 19, 'min_samples_leaf': 2, 'max_features': 0.4797239688856732}. Best is trial 47 with value: 0.9066666666666666.


🏃 View run optuna-hp-49-2025-03-27-10-09-57 at: http://localhost:5001/#/experiments/725460823759344054/runs/fc46a8360c564beb9bd506172d3d04b1
🧪 View experiment at: http://localhost:5001/#/experiments/725460823759344054


## 6. Register a model version using the best hyperparameters

In [5]:
from mlflow.models.signature import infer_signature

with mlflow.start_run(run_name=f"optuna-hp-final"):
    # Create the final model using the best hyperparameters
    final_model = RandomForestClassifier(**best_params)

    # Train the final model on the entire dataset
    final_model.fit(X, y)

    # Infer the model signature
    signature = infer_signature(X, final_model.predict(X))

    # Log the model to the "Models" section
    mlflow.sklearn.log_model(final_model, "random_forest_model", registered_model_name="random_forest_model", signature=signature)

Successfully registered model 'random_forest_model'.
2025/03/27 10:15:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random_forest_model, version 1


🏃 View run optuna-hp-final at: http://localhost:5001/#/experiments/725460823759344054/runs/65a85696dfd74f23bf99530341f553c9
🧪 View experiment at: http://localhost:5001/#/experiments/725460823759344054


Created version '1' of model 'random_forest_model'.


## 7. Retrieve a model version and use it for prediction

In [6]:
import mlflow.pyfunc

model_name = "random_forest_model"
model_version = 1

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

model.predict(X[:5])

array([0, 0, 0, 0, 0])